In [1]:
# clone github repo
#!git clone https://github.com/imics-lab/tts-gan.git

# move repo to cloud
#!mv tts-gan /content/drive/MyDrive/Colab\ Notebooks/tts-gan

# move to drive directory
%cd /content/drive/MyDrive/Colab Notebooks/tts-gan

#!mv tts-gan /content/drive/MyDrive/Colab\ Notebooks/tts-gan

!pip install einops torch torchvision einops matplotlib tqdm tsaug

[Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/tts-gan'
/content
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 14.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5

In [2]:
# mount to google drive
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# downloads the EEGMMIDB dataloader to TTS-GAN cloned repo

# convert ipynb into .py
#!jupyter nbconvert --to python "/content/drive/MyDrive/Colab Notebooks/tts-gan/EEGMMIDBDatasetLoaderV2.ipynb"

In [4]:
%cd /content/drive/MyDrive/Colab\ Notebooks/tts-gan
!ls

/content/drive/MyDrive/Colab Notebooks/tts-gan
 adamw.py			   LICENSE
 cfg.py				   LoadRealRunningJumping.py
 dataLoader.py			   LoadSyntheticRunningJumping.py
 EEGMMIDBDatasetLoaderV2.ipynb	   logs
 EEGMMIDBDatasetLoaderV2.py	   pre-trained-models
 EEGMMIDBDatasetToPickle.ipynb	   __pycache__
 functions.py			   README.md
 GANAugmentationTrainingV2.ipynb   RunningGAN_Train.py
 GAN_classwise_train		  'Running&JumpingVisualization.ipynb'
 GANModelGenerationTest.ipynb	   train_GAN_OG.py
 GANModels.py			   train_GAN.py
 images				   utils
 JumpingGAN_Train.py		   visualizationMetrics.py


In [5]:
# imports
from EEGMMIDBDatasetLoaderV2 import EEGMMIDBDataset
from torch.utils.data import DataLoader
from GANModels import Generator, Discriminator
from functions import train, copy_params, load_params, LinearLrDecay
from utils.utils import save_checkpoint
from torch.utils.tensorboard import SummaryWriter

import torch
import numpy as np
import os
import io
import matplotlib.pyplot as plt
from torchvision.transforms import ToTensor
import PIL.Image
from copy import deepcopy

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# path to save checkpoints
checkpoint_base_dir = "/content/drive/MyDrive/Colab Notebooks"

# create save directories
#all_classes = ['left_hand', 'right_hand', 'both_hands', 'both_feet', 'rest']
all_classes = ['left_hand', 'right_hand', 'rest']
checkpoint_paths = {}

for cls in classes:
  path = f'{checkpoint_base_dir}/EEGMMIDB_GAN_checkpoint/{cls}_GAN_checkpoint'
  os.makedirs(path, exist_ok=True)
  checkpoint_paths[cls] = path


# print("Checkpoint directories created:")
# print(checkpoint_paths)

In [7]:
!pip install tqdm

In [9]:
import sys
import train_GAN  # This is your train_GAN.py file
import os
from tqdm import tqdm
from glob import glob
import re


# Base configuration (edit as needed)
base_data_path = "/content/drive/MyDrive/eeg_data/eegmmidb_train_df.pkl"
checkpoint_root = "/content/drive/MyDrive/Colab Notebooks/EEGMMIDB_GAN_checkpoint"

# List of all classes to train
eeg_classes = ['left_hand', 'right_hand', 'both_hands', 'both_feet', 'rest']

num_epochs_per_batch = 50

# Shared hyperparameters
shared_args = {
    '--batch_size': '32',
    '--max_epoch': '50',
    '--max_iter': '3000',
    '--latent_dim': '100',
    '--channels': '64',
    '--img_size': '640',
    '--g_lr': '0.0002',
    '--d_lr': '0.0002',
    '--exp_name': '',  # placeholder, will be set per class
    '--class_name': '',  # placeholder, will be set per class
    '--data_path': base_data_path,
    '--grow_steps': '0'
}



for cls in tqdm(eeg_classes, desc='Training GAN per class'):
    print(f"\n=== 🔁 Starting class: {cls} ===")

    # Reset possible load_path from prior loop
    shared_args.pop('--load_path', None)

    clean_cls = cls.replace(" ", "_")
    save_dir = os.path.join(checkpoint_root, f"{clean_cls}_GAN_checkpoint")
    os.makedirs(save_dir, exist_ok=True)

    shared_args['--class_name'] = cls
    shared_args['--exp_name'] = f"GAN_{clean_cls}_exp"

    matching_logs = glob(f"/content/drive/MyDrive/Colab Notebooks/tts-gan/logs/GAN_{clean_cls}_exp_*")
    matching_logs = sorted(matching_logs, key=os.path.getmtime, reverse=True)

    if matching_logs:
        latest_log_dir = matching_logs[0]
        checkpoint_path = os.path.join(latest_log_dir, "Model", "checkpoint.pth")
        if os.path.exists(checkpoint_path):
            print(f"🔁 Resuming from checkpoint: {checkpoint_path}")
            shared_args['--load_path'] = checkpoint_path

    # Build sys.argv
    sys.argv = ['train_GAN.py']
    for k, v in shared_args.items():
        sys.argv.extend([k, v])

    # Train
    train_GAN.main()

Output hidden; open in https://colab.research.google.com to view.